In [2]:
import pandas as pd
import os
from datetime import datetime
import re
import matplotlib.pyplot as plt

In [3]:

#[INFO] [1715773048.812556027] [workspace_USB]: START	PID 3429	UID 1e790398-6269-4160-ae2e-cf1f0d9c667d	TIMESTAMP 1715773048506497093
#[INFO] [1715773049.277655391] [workspace_USB]: HAND	PID 3429	UID 1e790398-6269-4160-ae2e-cf1f0d9c667d	TIMESTAMP 1715773049274735592
#[INFO] [1715773049.303038248] [workspace_USB]: FINISH	PID 3429	UID 1e790398-6269-4160-ae2e-cf1f0d9c667d	TIMESTAMP 1715773049300404167

def process_log(log_file_path, remove_incomplete = False):
    # Read the log file and split it into lines
    with open(log_file_path, "r") as file:
        lines = file.readlines()

    # Initialize lists to store parsed data
    record_stamp_list = []
    pid_list = []
    uid_list = []
    timestamp_list = []
    
    # Iterate through lines, parsing the relevant data
    for i in range(0, len(lines)):
        line = lines[i].split()
        
        stamp = line[3]
        pid = line[5]
        uid = line[7]
        timestamp = int(line[9])
        
        record_stamp_list.append(stamp)
        pid_list.append(pid)
        uid_list.append(uid)
        timestamp_list.append(timestamp)        
    # Create a pandas DataFrame
    data = {
        "stamp": record_stamp_list,
        "pid": pid_list,
        "uid": uid_list,
        "timestamp": timestamp_list,
    }

    df = pd.DataFrame(data)
    # Convert timestamps to datetime objects
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.pivot_table(index=["pid","uid"], columns="stamp", values="timestamp", aggfunc='first').reset_index()
    if remove_incomplete:
        df = df.dropna()
    return df


In [5]:
def get_experiment_records(camera_path, controller_path, actuator_path, how="left"):
    cam = process_log(camera_path)
    controller = process_log(controller_path)
    actuator = process_log(actuator_path)
    
    merged_df = pd.merge(cam, controller, on="uid", how=how)
    return pd.merge(merged_df, actuator, on="uid", how=how)


In [11]:
# Constants for new columns
CAMERA_EXEC = "camera_exec"
ACTUATOR_REQ_EXEC = "actuator_req_exec"
WORKSPACE_REC_EXEC = "workspace_rec_exec"
ACTION_EXEC = "action_exec"
CAM_CTRL_TRANSMISSION = "camera_controller_transmission"
CTRL_ACT_TRANSMISSION = "camera_actuator_transmission"
LATENCY = "latency"
def get_mean_exec_times(df, drop = False):
    if drop:
        df = df.dropna()

    
    df[CAMERA_EXEC] = df["FINISH"] - df["START"]
    df[ACTUATOR_REQ_EXEC] = df["ACTUATE_END"] - df["ACTUATE_START"]
    df[CAM_CTRL_TRANSMISSION] = df["WORKSPACE_REC_START"] - df["FINISH"]
    df[WORKSPACE_REC_EXEC] = df["WORKSPACE_REC_END"] - df["WORKSPACE_REC_START"]
    #df[CTRL_ACT_TRANSMISSION] = df["ACTION_INIT"] - df["ACTUATE_END"]
    #df[ACTION_EXEC] = df["ACTION_FINISH"] - df["ACTION_INIT"]
    #df[LATENCY] = df["ACTION_INIT"] - df["START"]

    # Convert timedeltas to milliseconds as float
    df[CAMERA_EXEC] = df[CAMERA_EXEC].apply(lambda x: float(x.total_seconds() * 1000))
    df[ACTUATOR_REQ_EXEC] = df[ACTUATOR_REQ_EXEC].apply(lambda x: float(x.total_seconds() * 1000))
    df[CAM_CTRL_TRANSMISSION] = df[CAM_CTRL_TRANSMISSION].apply(lambda x: float(x.total_seconds() * 1000))
    df[WORKSPACE_REC_EXEC] = df[WORKSPACE_REC_EXEC].apply(lambda x: float(x.total_seconds() * 1000))
    #df[CTRL_ACT_TRANSMISSION] = df[CTRL_ACT_TRANSMISSION].apply(lambda x: float(x.total_seconds() * 1000)) 
    #df[ACTION_EXEC] = df[ACTION_EXEC].apply(lambda x: float(x.total_seconds() * 1000))
    #df[LATENCY] = df[LATENCY].apply(lambda x: float(x.total_seconds() * 1000))
    return df


In [14]:
hil_running = get_experiment_records("/home/szogabaha/src/uu/thesis/software/hilcps_edge_computing/experiments/hil_effect/hil_running/camera.log",
                                '/home/szogabaha/src/uu/thesis/software/hilcps_edge_computing/experiments/hil_effect/hil_running/controller.log',
                                '/home/szogabaha/src/uu/thesis/software/hilcps_edge_computing/experiments/hil_effect/hil_running/hil.log')
hil_not_running = get_experiment_records("/home/szogabaha/src/uu/thesis/software/hilcps_edge_computing/experiments/hil_effect/hil_not_running/camera.log",
                                '/home/szogabaha/src/uu/thesis/software/hilcps_edge_computing/experiments/hil_effect/hil_not_running/controller.log',
                                '/home/szogabaha/src/uu/thesis/software/hilcps_edge_computing/experiments/hil_effect/hil_not_running/hil.log')

df = get_mean_exec_times(hil_running, False)
print(df[[CAMERA_EXEC,ACTUATOR_REQ_EXEC, CAM_CTRL_TRANSMISSION, WORKSPACE_REC_EXEC]].mean())
df = get_mean_exec_times(hil_not_running, False)
print(df[[CAMERA_EXEC, ACTUATOR_REQ_EXEC, CAM_CTRL_TRANSMISSION, WORKSPACE_REC_EXEC]].mean())

stamp
camera_exec                       190.636148
actuator_req_exec                   5.516846
camera_controller_transmission      1.557096
workspace_rec_exec                  4.587134
dtype: float64
stamp
camera_exec                       186.460262
actuator_req_exec                   6.058496
camera_controller_transmission      2.482428
workspace_rec_exec                  4.248545
dtype: float64
